In [2]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.integrate import simps
import pywt

In [2]:
class Feature():

    def __init__(self, channels, readings, filename = "data.csv"):
        # name of file where data is being stored along with extension
        self.filename = filename
        self.channels = channels
        self.readings = readings
        self.delta_values = []
        self.theta_values = []
        self.alpha_values = []
        self.beta_values = []
        arr = range(self.readings)
        self.feature = pd.DataFrame(arr, columns=['Redundant'])
        self.feature.to_csv('feature.csv')

    # function plots welchs periodogram and calculates area under the curve for respective frequency band using composite simpsons rules
    def relative_psd(self):
        fs = 128
        t = 3
        # t is the duration of the signal, this value must be same is record.py and stimuli/cues.py as well

        # N = fs * t

        win_length = 4*fs
        # window length is normally 2/Lowest freq of interest which is 0.5 in our case thus the window length is 4 seconds

        data = pd.read_csv(self.filename)
        data.to_csv('data1.csv', header = False)
        data = pd.read_csv("data1.csv")
        # getting rid of garbage headers

        arr = range(self.readings)
        feature = pd.DataFrame(arr, columns=['Redundant'])
        feature.to_csv('feature.csv')
        

        for i in self.channels:

            indices = np.linspace(0, data[i].size, self.readings, dtype=int)
            k=0
            for j in range(self.readings) :
                freqs, psd = signal.welch(data[i].iloc[indices[k] : indices[k+1]], fs,)
                # nperseg=win_length

                """
                plt.plot(freqs, psd, color='k', lw=2)
                plt.xlabel('Frequency')
                plt.ylabel('Power spectral density')
                plt.title("Welch's periodogram")
                """

                # delta band 
                low = 0.5
                high = 4
                delta = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                delta_power = simps(psd[delta], dx=freq_res)

                total_power = simps(psd, dx=freq_res)

                delta_rel_power = delta_power / total_power
                self.delta_values.append(delta_rel_power)

                # print('relative delta power = %.3f '% delta_rel_power)

                # theta band 
                low = 4
                high = 8
                theta = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                theta_power = simps(psd[theta], dx=freq_res)
                theta_rel_power = theta_power / total_power
                # print('relative theta power = %.3f '% theta_rel_power)
                self.theta_values.append(theta_rel_power)

                # alpha band 
                low = 8
                high = 12
                alpha = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                alpha_power = simps(psd[alpha], dx=freq_res)
                alpha_rel_power = alpha_power / total_power
                # print('relative alpha power = %.3f '% alpha_rel_power)
                self.alpha_values.append(alpha_rel_power)

                # beta band 
                low = 12
                high = 30
                beta = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                beta_power = simps(psd[beta], dx=freq_res)
                beta_rel_power = beta_power / total_power
                # print('relative beta power = %.3f '% beta_rel_power)
                self.beta_values.append(beta_rel_power)
                k=k+1

            self.save_to_file(i, self.feature)
            self.delta_values = []
            self.theta_values = []
            self.alpha_values = []
            self.beta_values = []

        

        self.feature = self.feature.drop('Redundant', axis=1)
        self.feature.to_csv('feature.csv')

    def wavelet(self, channel):
        data = pd.read_csv(self.filename)
        sig = data[channel]
        dec=pywt.wavedec(sig, wavelet = 'db4', level=4)

    def save_to_file(self, channel, feature):
        feature[channel+'delta'] = self.delta_values
        feature[channel+'theta'] = self.theta_values
        feature[channel+'alpha'] = self.alpha_values
        feature[channel+'beta'] = self.beta_values

In [3]:
channels = ['EEG.AF3','EEG.F7','EEG.F3','EEG.FC5','EEG.T7','EEG.P7','EEG.O1','EEG.O2','EEG.P8','EEG.T8','EEG.FC6','EEG.F4','EEG.F8','EEG.AF4']

In [4]:
obj = Feature(channels)

In [5]:
obj.relative_psd()

relative delta power = 0.373 
relative theta power = 0.164 
relative alpha power = 0.149 
relative beta power = 0.172 
relative delta power = 0.410 
relative theta power = 0.133 
relative alpha power = 0.165 
relative beta power = 0.178 
relative delta power = 0.356 
relative theta power = 0.118 
relative alpha power = 0.218 
relative beta power = 0.140 
relative delta power = 0.429 
relative theta power = 0.127 
relative alpha power = 0.157 
relative beta power = 0.192 
relative delta power = 0.211 
relative theta power = 0.196 
relative alpha power = 0.147 
relative beta power = 0.198 
relative delta power = 0.254 
relative theta power = 0.165 
relative alpha power = 0.197 
relative beta power = 0.249 
relative delta power = 0.253 
relative theta power = 0.083 
relative alpha power = 0.353 
relative beta power = 0.215 
relative delta power = 0.191 
relative theta power = 0.061 
relative alpha power = 0.464 
relative beta power = 0.117 
relative delta power = 0.106 
relative theta pow

In [10]:
data = pd.read_csv('data.csv')
data.to_csv('data1.csv', header = False)
data = pd.read_csv("data1.csv")

In [26]:
data['EEG.AF3'].iloc[1:4]

1    4223.461426
2    4228.846191
3    4220.384766
Name: EEG.AF3, dtype: float64

In [22]:
indices = np.linspace(0, data['EEG.AF3'].size, 5,dtype=int)

In [24]:
indices[1]

162

In [27]:
data['EEG.AF3'].iloc[indices[0] : indices[1]]

0      4218.717773
1      4223.461426
2      4228.846191
3      4220.384766
4      4205.128418
          ...     
157    4180.512695
158    4178.717773
159    4187.307617
160    4189.743652
161    4187.051270
Name: EEG.AF3, Length: 162, dtype: float64

In [28]:
 df = pd.read_csv(r"C:/Users/KHALS/OneDrive/Desktop/professorX/prototype/sequence.csv")


In [31]:
markers = df["Sequence"].to_numpy()


In [32]:
name = 'EEG.AF3'

NameError: name 'df1' is not defined

In [36]:
arr = range(5)
df = pd.DataFrame(arr, columns=['Redundant'])
df.to_csv('feature.csv')

In [1]:
df.drop[]

SyntaxError: invalid syntax (<ipython-input-1-bb1f4eb13d59>, line 1)

In [6]:
        fs = 128
        t = 3
        # t is the duration of the signal, this value must be same is record.py and stimuli/cues.py as well

        # N = fs * t

        win_length = 4*fs
        # window length is normally 2/Lowest freq of interest which is 0.5 in our case thus the window length is 4 seconds

        data = pd.read_csv('data.csv')
        data.to_csv('data1.csv', header = False)
        data = pd.read_csv("data1.csv")
        # getting rid of garbage headers

        arr = range(5)
        feature = pd.DataFrame(arr, columns=['Redundant'])
        feature.to_csv('feature.csv')
        

        for i in self.channels:

            indices = np.linspace(0, data[i].size,5, dtype=int)
            k=0
            for j in range(self.readings) :
                freqs, psd = signal.welch(data[i].iloc[indices[k] : indices[k+1]], fs,)
                # nperseg=win_length

                """
                plt.plot(freqs, psd, color='k', lw=2)
                plt.xlabel('Frequency')
                plt.ylabel('Power spectral density')
                plt.title("Welch's periodogram")
                """

                # delta band 
                low = 0.5
                high = 4
                delta = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                delta_power = simps(psd[delta], dx=freq_res)

                total_power = simps(psd, dx=freq_res)

                delta_rel_power = delta_power / total_power
                self.delta_values.append(delta_rel_power)

                # print('relative delta power = %.3f '% delta_rel_power)

                # theta band 
                low = 4
                high = 8
                theta = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                theta_power = simps(psd[theta], dx=freq_res)
                theta_rel_power = theta_power / total_power
                # print('relative theta power = %.3f '% theta_rel_power)
                self.theta_values.append(theta_rel_power)

                # alpha band 
                low = 8
                high = 12
                alpha = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                alpha_power = simps(psd[alpha], dx=freq_res)
                alpha_rel_power = alpha_power / total_power
                # print('relative alpha power = %.3f '% alpha_rel_power)
                self.alpha_values.append(alpha_rel_power)

                # beta band 
                low = 12
                high = 30
                beta = np.logical_and(freqs >= low, freqs <= high)
                freq_res = freqs[1] - freqs[0] 
                beta_power = simps(psd[beta], dx=freq_res)
                beta_rel_power = beta_power / total_power
                # print('relative beta power = %.3f '% beta_rel_power)
                self.beta_values.append(beta_rel_power)
                k=k+1

            
            delta_values = []
            theta_values = []
            alpha_values = []
            beta_values = []

        

        self.feature = self.feature.drop('Redundant', axis=1)
        self.feature.to_csv('feature.csv')


NameError: name 'self' is not defined

In [26]:
indices = np.linspace(0, data['EEG.AF3'].size, 6, dtype=int)

In [27]:
data['EEG.AF3']

0      4218.717773
1      4223.461426
2      4228.846191
3      4220.384766
4      4205.128418
          ...     
644    4179.102539
645    4173.461426
646    4174.230957
647    4177.051270
648    4180.769043
Name: EEG.AF3, Length: 649, dtype: float64

In [30]:
data['EEG.AF3'].iloc[indices[4] : indices[5]]

519    4177.436035
520    4179.358887
521    4184.358887
522    4193.205078
523    4199.358887
          ...     
644    4179.102539
645    4173.461426
646    4174.230957
647    4177.051270
648    4180.769043
Name: EEG.AF3, Length: 130, dtype: float64

In [25]:
for i in range(5):
    print(i)

0
1
2
3
4
